# Gaussian Splatting Baseline

In [1]:
!nvidia-smi

Thu Jul  4 02:33:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   28C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
%load_ext autoreload
%autoreload 2

## Recursively Clone the Repo

In [ ]:
!git clone https://gradiently:$(gcloud secrets versions access 1 --secret="gh_pat")@github.com/gradiently/gaussian-splatting.git --recursive
!mkdir -p repos
!mv gaussian-splatting repos/gaussian-splatting

Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 734, done.
remote: Total 734 (delta 0), reused 0 (delta 0), pack-reused 734
Receiving objects: 100% (734/734), 2.13 MiB | 54.65 MiB/s, done.
Resolving deltas: 100% (429/429), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3157, done.        
remote: Counting objects: 100% (211/211), done.        
remote: Compressing objects: 100% (199/199), done.        
remote: Total 3157 (delta 101), reused 12 (delta 12), pack-reused 2946 (from 1)        
Receiving objects: 100% (3157/3157

## Installation

In [ ]:
!ls repos/gaussian-splatting/

arguments   environment.yml    LICENSE.md    README.md	SIBR_viewers  utils
assets	    full_eval.py       lpipsPyTorch  render.py	submodules
convert.py  gaussian_renderer  metrics.py    scene	train.py


In [3]:
import sys

In [4]:
sys.path.append("/content/repos/gaussian-splatting")

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/content/repos/gloss/src',
 '/content/repos/gaussian-splatting/submodules/diff-gaussian-rasterization',
 '/content/repos/gaussian-splatting/submodules/simple-knn',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/repos/gaussian-splatting']

In [ ]:
!pip install -q plyfile

### Install Submodules

In [ ]:
!pip install -e /content/repos/gaussian-splatting/submodules/diff-gaussian-rasterization

Obtaining file:///content/repos/gaussian-splatting/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
  Running setup.py develop for diff-gaussian-rasterization


In [ ]:
!pip install -e /content/repos/gaussian-splatting/submodules/simple-knn

Obtaining file:///content/repos/gaussian-splatting/submodules/simple-knn
  Preparing metadata (setup.py) ... done
  Running setup.py develop for simple-knn


## Download scenes

In [ ]:
!mkdir -p /content/gs2_test/
# !wget https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/datasets/input/tandt_db.zip -O /content/gs2_test/tandt_db.zip

--2024-07-03 19:14:34--  https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/datasets/input/tandt_db.zip
Resolving repo-sam.inria.fr (repo-sam.inria.fr)... 138.96.1.1
Connecting to repo-sam.inria.fr (repo-sam.inria.fr)|138.96.1.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 682628995 (651M) [application/zip]
Saving to: ‘/content/gs2_test/tandt_db.zip’

/content/gs2_test/t 100%[===================>] 651.00M  7.59MB/s    in 84s     

2024-07-03 19:16:00 (7.72 MB/s) - ‘/content/gs2_test/tandt_db.zip’ saved [682628995/682628995]



In [ ]:
!mkdir -p /content/gs2_test/hp
!gsutil -m rsync -r gs://ai-dust/gs_scenes/hp /content/gs2_test/hp

Unzip the asset

In [ ]:
# !cd /content/gs2_test && unzip tandt_db.zip > /dev/null && cd /content

In [ ]:
!ls /content/gs2_test/tandt/train

images	sparse	weights


## Importing

In [5]:
from PIL import Image
import sys
import numpy as np
from tqdm.auto import tqdm
from random import randint

import torch
import torchvision as tv

from scene import Scene
from scene.cameras import Camera
from gaussian_renderer import render, GaussianModel
from utils.general_utils import safe_state
from utils.loss_utils import l1_loss, ssim
from arguments import ModelParams, PipelineParams, OptimizationParams, get_combined_args, ParamGroup

In [6]:
def __init__(self, data, name : str, fill_none = False):
    self.data = data

def from_dict(self):
    for key, value in self.data.items():
        if key in vars(self) or f"_{key}" in vars(self):
            setattr(self, key, value)
    return self

def to_dict(self):
    data = vars(self)
    del data['data']
    return data

ParamGroup.__init__ = __init__
ParamGroup.from_dict = from_dict
ParamGroup.to_dict = to_dict

In [7]:
from pathlib import Path

In [8]:
SOURCE_PATH = Path("/content/gs2_test/tandt/truck")
(SOURCE_PATH/"weights").mkdir(exist_ok=True, parents=True)

In [9]:
device = "cuda"

In [27]:
INPUT_ARGS = {
    'iterations':90000,
    'ip': "127.0.0.1",
    'port': 6009,
    'debug_from': -1,
    'detect_anomaly': False,
    'test_iterations': [7_000, 30_000],
    'save_iterations': [7_000, 30_000],
    'quiet': True,
    'checkpoint_iterations': [],
    'data_device':device,
    'start_checkpoint': None,
    'model_path': str(SOURCE_PATH/"weights"),
    'images': str(SOURCE_PATH / "images"),
    'source_path': str(SOURCE_PATH),
    'resolution':1,
}
mp = ModelParams(INPUT_ARGS).from_dict()
pp = PipelineParams(INPUT_ARGS).from_dict()
op = OptimizationParams(INPUT_ARGS).from_dict()

### Print current parameters

In [28]:
mp.to_dict()

{'sh_degree': 3,
 '_source_path': '',
 '_model_path': '',
 '_images': 'images',
 '_resolution': -1,
 '_white_background': False,
 'data_device': 'cuda',
 'eval': False,
 'model_path': '/content/gs2_test/tandt/truck/weights',
 'images': '/content/gs2_test/tandt/truck/images',
 'source_path': '/content/gs2_test/tandt/truck',
 'resolution': 1}

In [29]:
pp.to_dict()

{'convert_SHs_python': False, 'compute_cov3D_python': False, 'debug': False}

In [30]:
op.to_dict()

{'iterations': 90000,
 'position_lr_init': 0.00016,
 'position_lr_final': 1.6e-06,
 'position_lr_delay_mult': 0.01,
 'position_lr_max_steps': 30000,
 'feature_lr': 0.0025,
 'opacity_lr': 0.05,
 'scaling_lr': 0.005,
 'rotation_lr': 0.001,
 'percent_dense': 0.01,
 'lambda_dssim': 0.2,
 'densification_interval': 100,
 'opacity_reset_interval': 3000,
 'densify_from_iter': 500,
 'densify_until_iter': 15000,
 'densify_grad_threshold': 0.0002,
 'random_background': False}

### Load gaussians & scene

In [31]:
gaussians = GaussianModel(mp.sh_degree)

In [32]:
scene = Scene(mp, gaussians)

Reading camera 251/251
Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  136029


In [33]:
gaussians.training_setup(op)

In [34]:
scene

In [35]:
WHITE_BACKGROUND = True

In [36]:
bg_color = [1, 1, 1] if WHITE_BACKGROUND else [0, 0, 0]
background = torch.tensor(bg_color, dtype=torch.float32, device=device)

In [37]:
@torch.no_grad
def densification(
    idx,
    op,
    gaussians,
    scene,
    visibility_filter,
    radii,
    min_opacity: float = 0.005,
  ):
  if idx < op.densify_until_iter:
      gaussians.max_radii2D[visibility_filter] = torch.max(gaussians.max_radii2D[visibility_filter], radii[visibility_filter])
  if idx > op.densify_from_iter and idx % op.densification_interval == 0:
      size_threshold = 20 if idx > op.opacity_reset_interval else None
      gaussians.densify_and_prune(
          op.densify_grad_threshold,
          min_opacity,
          scene.cameras_extent,
          size_threshold)
  if idx % op.opacity_reset_interval == 0 or (WHITE_BACKGROUND and idx == op.densify_from_iter):
      gaussians.reset_opacity()

In [38]:
viewpoint_stack = None
ema_loss_for_log = 0.0

## Training loop

In [39]:
pg_bar = tqdm(range(op.iterations),leave=False, desc="🚀 training")

for idx in pg_bar:
    gaussians.update_learning_rate(idx)
    # Every 1000 its we increase the levels of SH up to a maximum degree
    if idx % 1000 == 0:
        gaussians.oneupSHdegree()
    # Pick a random Camera
    if not viewpoint_stack:
        viewpoint_stack = scene.getTrainCameras().copy()
    viewpoint_cam = viewpoint_stack.pop(randint(0, len(viewpoint_stack)-1))

    # setup background, considering if switching in random background
    bg = torch.rand((3), device=device) if op.random_background else background

    # this is where the magic happens
    #
    render_pkg = render(viewpoint_cam, gaussians, pp, bg)

    # unpack the render package
    image = render_pkg["render"]
    viewspace_point_tensor = render_pkg["viewspace_points"]
    visibility_filter = render_pkg["visibility_filter"]
    radii = render_pkg["radii"]

    # ground truth image
    gt_image = viewpoint_cam.original_image.to(device)

    Ll1 = l1_loss(image, gt_image)
    loss = (1.0 - op.lambda_dssim) * Ll1 + op.lambda_dssim * (1.0 - ssim(image, gt_image))
    loss.backward()

    gaussians.optimizer.step()
    gaussians.optimizer.zero_grad(set_to_none = True)

    densification(idx, op, gaussians, scene, visibility_filter, radii)

    with torch.no_grad():
        ema_loss_for_log = 0.4 * loss.item() + 0.6 * ema_loss_for_log
        if idx % 10 == 0:
            pg_bar.set_postfix({"loss": f"{ema_loss_for_log:.{7}f}"})

🚀 training:   0%|          | 0/90000 [00:00<?, ?it/s]

In [40]:
save_path = str(Path(scene.model_path)/f"chkpnt_{idx:06d}.pth")
torch.save((gaussians.capture(), idx), save_path)

In [41]:
scene.save(idx)

In [25]:
for i, plyfile in enumerate(Path(scene.model_path).rglob("point_cloud.ply")):
    print(f"gsutil cp {plyfile} gs://ai-dust/gs_scenes/plyfiles/hp{i:02d}.ply")

gsutil cp /content/gs2_test/tandt/truck/weights/point_cloud/iteration_89999/point_cloud.ply gs://ai-dust/gs_scenes/plyfiles/hp00.ply
gsutil cp /content/gs2_test/tandt/truck/weights/point_cloud/iteration_29999/point_cloud.ply gs://ai-dust/gs_scenes/plyfiles/hp01.ply


In [42]:
!gsutil cp /content/gs2_test/tandt/truck/weights/point_cloud/iteration_89999/point_cloud.ply gs://ai-dust/gs_scenes/plyfiles/hp00.ply

Copying file:///content/gs2_test/tandt/truck/weights/point_cloud/iteration_89999/point_cloud.ply [Content-Type=application/octet-stream]...
\
Operation completed over 1 objects/1.7 MiB.                                      


In [111]:
!gsutil cp /content/gs2_test/hp/weights/point_cloud/iteration_4999/point_cloud.ply gs://ai-dust/gs_scenes/plyfiles/hp01.ply


Copying file:///content/gs2_test/hp/weights/point_cloud/iteration_4999/point_cloud.ply [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/704.1 KiB.                                    
